### Crowdvariant Analysis - Machine Learning [All Technologies]
<br>

**Summary**

1. Data collection and data cleaning
2. Data preprocessing
3. Machine Learning analysis

** Notes **

- Gathered crowdsourced labels from the crowdvariant study
- high confidence labels only available for HG002
    - Are there other labels?
- All deletions
- 1514 data points total
    - 552 Heterozygous Variant (CrowdVar Label = 1)  [Confidence: >=84%]
    - 959 Homozygous Variant (CrowdVar Label = 0)    [Confidence: >=84%]
    - 3   Homozygous Reference (CrowdVar Label = 2)  [Confidence: >=91%]
    - 1   Unknown

***
Data 
***

**Train/Test Data**

- CrowdVariant Data - cleaned and parsed in Part 1



** Prediction Dataset **

- HG002 Deletions
- Randomly selected datapoints (Try 1) April 2017

***
Machine Learning
***

**Objective**
- Train [Random Forest Classifier](http://scikit-learn.org/stable/modules/ensemble.html#forest) with labeled CrowdVariant Data
- Train model using train test split [train on 70% of data and test on 30%]
- Predict Genotype Labels in the 5000 randomly selected set of deletions from svviz
- [Multiclass Classifiers](http://scikit-learn.org/stable/modules/multiclass.html)

**Future Tasks:** 
1. Train model using K-Fold validation
2. Run the model multiple times and take an average of the precision score
2. Compare this precision score to the average precision score of another model [i.e.: neural net]

In [1]:
"""
Imports
"""
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from scipy.stats import ks_2samp
from scipy import stats
from matplotlib import pyplot
from sklearn import preprocessing
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
import sqlite3
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import plotly.plotly as py
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, precision_score
from sklearn import preprocessing
from ggplot import *
from bokeh.charts import TimeSeries
from bokeh.models import HoverTool
from bokeh.plotting import show
from bokeh.charts import Scatter, Histogram, output_file, show
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.charts import Bar, output_file, show
import bokeh.palettes as palettes
from bokeh.models import HoverTool, BoxSelectTool, Legend
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



In [2]:
### Import Data
df_crowd = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/Train/Final_DF/CrowdVar.Train_HG002.csv')

In [3]:
### Copy data in new dataframe as a later reference
df_crowd_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/Train/Final_DF/CrowdVar.Train_HG002.csv')

In [4]:
df_crowd.head(3)

,chrom,start,end,sample,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,...,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label,CN0_prob,CN1_prob,CN2_prob
0,1,187464828,187466479,HG002,579.446154,17.934094,65.0,756.430769,163.879409,0.0,...,8,0.096911,0,0.0,0,0,1.0,0.00,0.91,0.09
1,1,33156824,33157000,HG002,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,...,2,0.221591,0,0.0,0,0,1.0,0.04,0.91,0.05
2,1,53594099,53595428,HG002,574.335366,18.613946,164.0,678.518293,139.056203,31.0,...,3,0.059443,0,0.0,0,0,0.0,0.96,0.04,0.00


In [5]:
### Drop irrelevant columns and GT information
df_crowd.drop(['GTcons'], axis=1, inplace = True)
df_crowd.drop(['GTconflict'], axis=1, inplace = True)
df_crowd.drop(['GTsupp'], axis=1, inplace = True)
# df_crowd.drop('SVtype', axis=1)
# df_crowd.drop(['type'],axis=1)
df_crowd.drop(['start'],axis=1, inplace = True)
df_crowd.drop(['end'],axis=1, inplace = True)
df_crowd.drop(['chrom'],axis=1, inplace = True)
# df_crowd.drop('Size',axis=1)
df_crowd.drop(['CN0_prob'],axis=1, inplace = True)
df_crowd.drop(['CN1_prob'],axis=1, inplace = True)
df_crowd.drop(['CN2_prob'],axis=1, inplace = True)
df_crowd.drop(['TenX.GT'],axis=1, inplace = True)
df_crowd.drop(['pacbio.GT'],axis=1, inplace = True)
df_crowd.drop(['IllMP.GT'],axis=1, inplace = True)
df_crowd.drop(['Ill250.GT'],axis=1, inplace = True)
df_crowd.drop(['Ill300x.GT'],axis=1, inplace = True)
df_crowd.drop(['sample'],axis=1, inplace = True)

In [6]:
#Save new dataframe to csv file [with dropped columns] and store header names
df_crowd.to_csv('df_crowd_headers.csv', index=False)

In [7]:
df_crowd.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,2.0,0.0,1651,8,0.096911,0,0.0,0,0,1.0
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,12.0,0.0,176,2,0.221591,0,0.0,0,0,1.0
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,3.0,0.0,1329,3,0.059443,0,0.0,0,0,0.0


In [8]:
# Store data in a new variable which will be converted to a matrix
X = df_crowd

In [9]:
X.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,2.0,0.0,1651,8,0.096911,0,0.0,0,0,1.0
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,12.0,0.0,176,2,0.221591,0,0.0,0,0,1.0
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,3.0,0.0,1329,3,0.059443,0,0.0,0,0,0.0


** Impute missing values using KNN **

Used 3 of the nearest neighbors to impute missing values

In [10]:
# Convert dataframe to matrix
X=X.as_matrix()

#Imput missing values from three closest observations
X_imputed=KNN(k=3).complete(X)
X=pd.DataFrame(X_imputed)

Imputing row 1/1515 with 0 missing, elapsed time: 1.802
Imputing row 101/1515 with 0 missing, elapsed time: 1.849
Imputing row 201/1515 with 0 missing, elapsed time: 1.851
Imputing row 301/1515 with 0 missing, elapsed time: 1.855
Imputing row 401/1515 with 0 missing, elapsed time: 1.859
Imputing row 501/1515 with 0 missing, elapsed time: 1.860
Imputing row 601/1515 with 0 missing, elapsed time: 1.863
Imputing row 701/1515 with 0 missing, elapsed time: 1.863
Imputing row 801/1515 with 0 missing, elapsed time: 1.865
Imputing row 901/1515 with 0 missing, elapsed time: 1.865
Imputing row 1001/1515 with 0 missing, elapsed time: 1.866
Imputing row 1101/1515 with 0 missing, elapsed time: 1.870
Imputing row 1201/1515 with 0 missing, elapsed time: 1.872
Imputing row 1301/1515 with 0 missing, elapsed time: 1.873
Imputing row 1401/1515 with 0 missing, elapsed time: 1.875
Imputing row 1501/1515 with 0 missing, elapsed time: 1.875


In [11]:
# Add headers to the data frame with newly imputed missing values [FYI: header removed during imputation]
X.columns=['Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_insertSizeScore_insertSizeScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','size','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','Label']

In [12]:
X.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,2.0,0.0,1651.0,8.0,0.096911,0.0,0.0,0.0,0.0,1.0
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,12.0,0.0,176.0,2.0,0.221591,0.0,0.0,0.0,0.0,1.0
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,3.0,0.0,1329.0,3.0,0.059443,0.0,0.0,0.0,0.0,0.0


In [13]:
# Store Labels in a new 'Y' DataFrame
Y = pd.DataFrame()
Y['Label'] = X['Label']
#Y = X.pop('Label')

In [14]:
#Count the number of labels
pd.value_counts(Y['Label'].values, sort=False)

 1.0    959
 0.0    552
-1.0      1
 2.0      3
dtype: int64

**Select Features to Train Machine Learning Model**
- The features in the CrowdVar dataframe must match all of the features in the svviz dataframe
- The next step selects all of the features found in both the CrowdVar dataframe and the svviz dataframe

In [15]:
#Features in training set must match features in the prediction set
X=X[['Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct']]

In [16]:
X.head()

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,TenX.HP2_ref_insertSize_mean,TenX.HP2_ref_insertSize_std,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,321.500000,25.500000,2.0,0.0,8.0,0.096911,0.0,0.0,0.0,0.0
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,472.250000,138.336019,12.0,0.0,2.0,0.221591,0.0,0.0,0.0,0.0
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,988.000000,181.532366,3.0,0.0,3.0,0.059443,0.0,0.0,0.0,0.0
3,582.000000,12.928956,38.0,697.026316,152.204932,4.0,34.0,0.0,528.007583,93.211521,...,330.142857,90.760168,13.0,1.0,1.0,0.240310,0.0,0.0,0.0,0.0
4,577.937500,16.320496,80.0,751.150000,128.707527,0.0,80.0,0.0,528.103762,93.192739,...,627.210526,409.833031,19.0,0.0,1.0,0.046729,0.0,0.0,0.0,0.0


In [17]:
X_df = pd.DataFrame(X)
X_df.to_csv('X_df.csv', index=False)

** Standardize Data **

** Note **

Features only were standardized

** For final analysis, data not standardized **

In [18]:
# scaler=preprocessing.StandardScaler()
# X=scaler.fit_transform(X)

** Train Random Forest Classifier **

In [19]:
# Train Test Split
# Train on 70% of the data and test on 30%
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [20]:
# Train Random Forest Classifier
model = RandomForestClassifier() 
model.fit(X_train, y_train)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

[Save Classifier](https://stackoverflow.com/questions/10017086/save-naive-bayes-trained-classifier-in-nltk)

In [21]:
# Save Trained Classifier for Individual Technologies
import pickle
f = open('my_classifier.pickle', 'wb')
pickle.dump(model, f)
f.close()

In [22]:
#NOTE: Training Set - Show number of Hom Ref, Hom Var, Het Var datapoints the model was trained on
ytrain = pd.DataFrame()
ytrain['ytrain'] = y_train
pd.value_counts(ytrain['ytrain'].values, sort=False)

 1.0    671
 0.0     73
-1.0      1
 2.0      2
dtype: int64

** Precision Score **
- Overall model performance
- Using 30% of original dataset (test set)
- Truth labels: CrowdVariant labels

In [23]:
model.predict(X_test)

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
        0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,
        1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,
        1.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0

In [24]:
pred = model.predict(X_test)

In [25]:
precision_score(pred, y_test, average='micro') 

0.98901098901098905

In [26]:
# Add original labels and predicted labels back to the original dataframe
df_Xtest = pd.DataFrame(X_test)
df_Xtest.head()

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,TenX.HP2_ref_insertSize_mean,TenX.HP2_ref_insertSize_std,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct
1222,589.435606,7.274880,528.0,633.607955,149.863131,243.0,285.0,0.0,541.900846,90.303799,...,0.000000,0.000000,0.0,0.0,1.0,0.12500,0.0,0.000000,0.0,0.0
310,584.644269,9.538767,253.0,635.561265,162.372800,127.0,126.0,0.0,529.934868,92.347999,...,351.789474,112.498926,38.0,0.0,1.0,0.10625,0.0,0.000000,0.0,0.0
9,567.346939,21.428144,49.0,716.285714,171.177077,0.0,30.0,19.0,549.786572,74.408848,...,456.076923,157.563888,13.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0
785,581.916168,5.442583,167.0,599.826347,170.450582,162.0,5.0,0.0,532.700078,91.693132,...,336.682927,104.134397,41.0,0.0,1.0,0.31405,1.0,0.404959,0.0,0.0
295,577.156028,17.312843,141.0,609.879433,148.310508,74.0,67.0,0.0,520.960245,91.350233,...,0.000000,0.000000,0.0,0.0,2.0,0.16358,0.0,0.000000,0.0,0.0


In [27]:
df_Xtest.columns=['Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct']

In [28]:
labels = pd.DataFrame(y_test)

In [29]:
df_Xtest['predicted_label'] = pred
df_Xtest['Label'] = df_crowd['Label']
df_Xtest['chrom'] = df_crowd_2['chrom']
df_Xtest['start'] = df_crowd_2['start']
df_Xtest['end'] = df_crowd_2['end']
# df_Xtest['Y_test'] = labels

In [30]:
df_Xtest.head()

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,predicted_label,Label,chrom,start,end
1222,589.435606,7.274880,528.0,633.607955,149.863131,243.0,285.0,0.0,541.900846,90.303799,...,0.12500,0.0,0.000000,0.0,0.0,0.0,0.0,2,82340460,82340780
310,584.644269,9.538767,253.0,635.561265,162.372800,127.0,126.0,0.0,529.934868,92.347999,...,0.10625,0.0,0.000000,0.0,0.0,1.0,1.0,13,29722595,29722915
9,567.346939,21.428144,49.0,716.285714,171.177077,0.0,30.0,19.0,549.786572,74.408848,...,0.00000,0.0,0.000000,0.0,0.0,1.0,1.0,1,246877707,246877940
785,581.916168,5.442583,167.0,599.826347,170.450582,162.0,5.0,0.0,532.700078,91.693132,...,0.31405,1.0,0.404959,0.0,0.0,1.0,1.0,5,120727436,120727557
295,577.156028,17.312843,141.0,609.879433,148.310508,74.0,67.0,0.0,520.960245,91.350233,...,0.16358,0.0,0.000000,0.0,0.0,1.0,1.0,12,105773513,105773837


In [31]:
pd.value_counts(df_Xtest['Label'].values, sort=False)

0.0    166
1.0    288
2.0      1
dtype: int64

In [32]:
pd.value_counts(df_Xtest['predicted_label'].values, sort=False)

0.0    164
1.0    291
dtype: int64

In [33]:
from sklearn.metrics import confusion_matrix
ytest = df_Xtest['Label']
predict = df_Xtest['predicted_label']
print(confusion_matrix(ytest, predict))

[[163   3   0]
 [  1 287   0]
 [  0   1   0]]


In [34]:
pd.crosstab(ytest, predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0.0,163,3,166
1.0,1,287,288
2.0,0,1,1
All,164,291,455


In [35]:
# import pylab as pl
# labels = ['-1', '0', '1', '2']
# cm = confusion_matrix(y_test, pred)
# print(cm)
# fig = plt.figure()
# ax = fig.add_subplot(111)
# cax = ax.matshow(cm)
# pl.title('Confusion matrix of the classifier')
# fig.colorbar(cax)
# ax.set_xticklabels([''] + labels)
# ax.set_yticklabels([''] + labels)
# pl.xlabel('Predicted')
# pl.ylabel('True')
# pl.show()

In [36]:
df_Xtest.to_csv('df_Xtest.csv', index=False)

** Future Task **

Use GridSearch to find the best parameters

***
Prediction 
***

Used svviz.HG002 data from first 5000 random selection

**Data Label Update**

Svviz GT Labels and CrowdVar GT Labels do not match

Created a new set of labels for the svviz HG002 Deletions Dataframe

CrowdVar GT Label Key

- 0: Hom. Var.
- 1: Het Var
- 2: Hom Ref

svviz GTcons Labels

- 0: Hom Ref
- 1: Het Var
- 2: Hom Var

Changed svviz DF (svviz.Annotate.DEL.HG002_2.csv) to have the following data labels (which match CrowdVar)
New Column (GIAB_Crowd)

- 0: Hom. Var.
- 1: Het Var
- 2: Hom Ref

**Stored** all rows that have '-1' in a separate dataframe
- including '-1' will throw an error in accuracy and prediction measures
- will use these entries as final prediction set

Used excel formula to manually change svviz.HG002 labels

=IF(GB2=0,2,IF(GB2=2,0,1))

In [90]:
# Read in HG002 DEL dataframe
HG002_pred = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/svviz.Annotate.DEL.HG002_data2.csv')

In [91]:
HG002_pred_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/svviz.Annotate.DEL.HG002_data2.csv')

In [92]:
### Drop irrelevant columns
# Note: Features in the prediction dataframe must match the features in the dataframe (CrowdVar) used to train the RF model
# NOTE TODO: Include size
HG002_pred.drop(['GTcons'], axis=1, inplace = True)
HG002_pred.drop(['GTconflict'], axis=1, inplace = True)
HG002_pred.drop(['GTsupp'], axis=1, inplace = True)
HG002_pred.drop('SVtype', axis=1)
HG002_pred.drop('type',axis=1)
HG002_pred.drop(['type'],axis=1, inplace = True)
HG002_pred.drop(['SVtype'],axis=1, inplace = True)
HG002_pred.drop(['start'],axis=1, inplace = True)
HG002_pred.drop(['end'],axis=1, inplace = True)
HG002_pred.drop(['chrom'],axis=1, inplace = True)
HG002_pred.drop(['Size'],axis=1, inplace = True)
HG002_pred.drop(['TenX.GT'],axis=1, inplace = True)
HG002_pred.drop(['pacbio.GT'],axis=1, inplace = True)
HG002_pred.drop(['IllMP.GT'],axis=1, inplace = True)
HG002_pred.drop(['Ill250.GT'],axis=1, inplace = True)
HG002_pred.drop(['Ill300x.GT'],axis=1, inplace = True)
HG002_pred.drop(['sample'],axis=1, inplace = True)
HG002_pred.drop(['id'],axis=1, inplace = True)
# HG002_pred.drop(['GIAB_Crowd'],axis=1, inplace = True)

In [93]:
HG002_pred.head()

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,pacbio.ref_insertSize_mean,pacbio.ref_insertSize_std,pacbio.ref_reason_alignmentScore,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,541.033679,91.702755,...,9694.425532,4306.492796,47.0,0,0.0,0,0.000000,0,0,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,520.882022,78.521682,...,9218.592593,4009.865909,27.0,0,0.0,0,0.000000,0,0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,531.881724,77.118761,...,10296.150000,4547.097621,40.0,0,0.0,1,0.675712,0,0,2
3,587.5,4.5,2.0,218.5,20.5,2.0,0.0,0.0,539.138453,85.428725,...,10774.622640,4040.498761,53.0,0,0.0,0,0.000000,0,0,2
4,592.0,0.0,1.0,1009.0,0.0,0.0,1.0,0.0,514.035095,90.478044,...,10438.000000,4823.436414,19.0,0,0.0,0,0.000000,0,0,2


In [94]:
HG002_pred.to_csv('HG002_pred.csv', index=False)

In [95]:
X2 = HG002_pred

** Impute missing values using KNN **

In [96]:
# Convert dataframe to matrix
X2=X2.as_matrix()

#Imput missing values from three closest observations
X2_imputed=KNN(k=3).complete(X2)
X2=pd.DataFrame(X2_imputed)

Imputing row 1/2805 with 1 missing, elapsed time: 6.909
Imputing row 101/2805 with 2 missing, elapsed time: 6.920
Imputing row 201/2805 with 1 missing, elapsed time: 6.935
Imputing row 301/2805 with 1 missing, elapsed time: 6.949
Imputing row 401/2805 with 1 missing, elapsed time: 6.959
Imputing row 501/2805 with 1 missing, elapsed time: 6.964
Imputing row 601/2805 with 2 missing, elapsed time: 6.975
Imputing row 701/2805 with 1 missing, elapsed time: 6.984
Imputing row 801/2805 with 1 missing, elapsed time: 6.989
Imputing row 901/2805 with 2 missing, elapsed time: 6.995
Imputing row 1001/2805 with 1 missing, elapsed time: 7.004
Imputing row 1101/2805 with 1 missing, elapsed time: 7.012
Imputing row 1201/2805 with 2 missing, elapsed time: 7.018
Imputing row 1301/2805 with 2 missing, elapsed time: 7.027
Imputing row 1401/2805 with 1 missing, elapsed time: 7.033
Imputing row 1501/2805 with 1 missing, elapsed time: 7.040
Imputing row 1601/2805 with 1 missing, elapsed time: 7.056
Imputing 

In [97]:
# Rename Columns in the dataframe
X2.columns = ['Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_insertSizeScore_orientation','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_alignmentScore','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_insertSizeScore_insertSizeScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_insertSizeScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_insertSizeScore_insertSizeScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_insertSizeScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','GIAB_Crowd']

In [98]:
# Select columns that match columns in training dataframe
#Features in training set must match features in the prediction set
X2=X2[['Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct']]

In [99]:
X2.to_csv('X2.csv', index=False)

** Standardize Data **

In [100]:
# scaler=preprocessing.StandardScaler()
# X=scaler.fit_transform(X2)

In [101]:
model.predict(X)

array([ 1.,  1.,  0., ...,  0.,  0.,  0.])

** Precision Score **

Data: 5000 randomly selected datapoints (Round 1) svviz GT vs. model selected GT from trained RF model

The ratio of tp /(tp +fn)

Intuitively the ability of a model to avoid labeling negative events as positive events

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

In [102]:
model.predict_proba(X)

array([[ 0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0.1,  0.9,  0. ],
       [ 0. ,  0.9,  0.1,  0. ],
       ..., 
       [ 0. ,  1. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ]])

In [103]:
pred = model.predict_proba(X)

In [104]:
X6 = pd.concat([X2, pd.DataFrame(pred, columns=['1','2','3','4'])])

In [113]:
X6.shape

(2805, 174)

In [114]:
X6.to_csv('X6_2_.csv', index=False)

In [118]:
X6 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant_Analysis/X6_2_.csv')

In [119]:
X6.shape

(1515, 174)

In [120]:
X6.head()

,1,2,3,4,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,pacbio.ref_count,pacbio.ref_insertSize_mean,pacbio.ref_insertSize_std,pacbio.ref_reason_alignmentScore,refN_cnt,refN_pct,segdup_cnt,segdup_pct,tandemrep_cnt,tandemrep_pct
0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0,...,47.0,9694.425532,4306.492796,47.0,0,0,0,0.000000,0,0.0
1,0.0,0.1,0.9,0.0,0.0,0.0,0,0.0,0.0,0,...,27.0,9218.592593,4009.865909,27.0,0,0,0,0.000000,0,0.0
2,0.0,0.9,0.1,0.0,0.0,0.0,0,0.0,0.0,0,...,40.0,10296.150000,4547.097621,40.0,0,0,1,0.675712,0,0.0
3,0.0,0.0,1.0,0.0,819.0,0.0,1,253.0,0.0,1,...,53.0,10774.622640,4040.498761,53.0,0,0,0,0.000000,0,0.0
4,0.0,0.9,0.1,0.0,0.0,0.0,0,0.0,0.0,0,...,19.0,10438.000000,4823.436414,19.0,0,0,0,0.000000,0,0.0


In [121]:
X6.shape

(1515, 174)

In [122]:
X6['GIAB_Crowd'] = HG002_pred['GIAB_Crowd']
# X6['model_pred_label'] = model.predict(X)
X6['chrom'] = HG002_pred_2['chrom']
X6['start'] = HG002_pred_2['start']
X6['end'] = HG002_pred_2['end']
X6['Size'] = HG002_pred_2['Size']
X6['GTcons'] = HG002_pred_2['GTcons']
X6['Ill250.GT'] = HG002_pred_2['Ill250.GT']
X6['Ill300x.GT'] = HG002_pred_2['Ill300x.GT']
X6['IllMP.GT'] = HG002_pred_2['IllMP.GT']
X6['pacbio.GT'] = HG002_pred_2['pacbio.GT']

In [123]:
X6.shape

(1515, 184)

In [124]:
X6.head()

,1,2,3,4,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,GIAB_Crowd,chrom,start,end,Size,GTcons,Ill250.GT,Ill300x.GT,IllMP.GT,pacbio.GT
0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0,...,2,10,21929089,21929235,-145,0,0,0.0,0.0,0.0
1,0.0,0.1,0.9,0.0,0.0,0.0,0,0.0,0.0,0,...,2,17,744621,744963,-231,0,0,0.0,0.0,0.0
2,0.0,0.9,0.1,0.0,0.0,0.0,0,0.0,0.0,0,...,2,21,32114178,32114915,-648,0,0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,819.0,0.0,1,253.0,0.0,1,...,2,16,5702979,5703571,-230,0,0,0.0,0.0,0.0
4,0.0,0.9,0.1,0.0,0.0,0.0,0,0.0,0.0,0,...,2,5,168173265,168174026,-215,0,0,0.0,0.0,0.0


In [125]:
X6.to_csv('X6_all_.csv', index=False)

In [126]:
X6.rename(columns={'1': 'unknown'}, inplace=True)
X6.rename(columns={'2': 'Hom_Var'}, inplace=True)
X6.rename(columns={'3': 'Het_Var'}, inplace=True)
X6.rename(columns={'4': 'Hom_Ref'}, inplace=True)

In [127]:
X6.head()

,unknown,Hom_Var,Het_Var,Hom_Ref,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,GIAB_Crowd,chrom,start,end,Size,GTcons,Ill250.GT,Ill300x.GT,IllMP.GT,pacbio.GT
0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0,...,2,10,21929089,21929235,-145,0,0,0.0,0.0,0.0
1,0.0,0.1,0.9,0.0,0.0,0.0,0,0.0,0.0,0,...,2,17,744621,744963,-231,0,0,0.0,0.0,0.0
2,0.0,0.9,0.1,0.0,0.0,0.0,0,0.0,0.0,0,...,2,21,32114178,32114915,-648,0,0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,819.0,0.0,1,253.0,0.0,1,...,2,16,5702979,5703571,-230,0,0,0.0,0.0,0.0
4,0.0,0.9,0.1,0.0,0.0,0.0,0,0.0,0.0,0,...,2,5,168173265,168174026,-215,0,0,0.0,0.0,0.0


In [128]:
predicted = pd.DataFrame()
predicted['model_pred_label'] = model.predict(X)
X6['model_pred_label'] = predicted['model_pred_label']

In [129]:
X6=X6[['chrom','start','end','Size','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','refN_cnt','refN_pct','segdup_cnt','segdup_pct','tandemrep_cnt','tandemrep_pct','Ill250.GT','Ill300x.GT','IllMP.GT','pacbio.GT','GIAB_Crowd','GTcons','model_pred_label','Hom_Var','Het_Var','Hom_Ref','unknown']]

In [130]:
X6.head()

,chrom,start,end,Size,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,Ill300x.GT,IllMP.GT,pacbio.GT,GIAB_Crowd,GTcons,model_pred_label,Hom_Var,Het_Var,Hom_Ref,unknown
0,10,21929089,21929235,-145,0.0,0.0,0,0.0,0.0,0,...,0.0,0.0,0.0,2,0,1.0,0.0,1.0,0.0,0.0
1,17,744621,744963,-231,0.0,0.0,0,0.0,0.0,0,...,0.0,0.0,0.0,2,0,1.0,0.1,0.9,0.0,0.0
2,21,32114178,32114915,-648,0.0,0.0,0,0.0,0.0,0,...,0.0,0.0,0.0,2,0,0.0,0.9,0.1,0.0,0.0
3,16,5702979,5703571,-230,819.0,0.0,1,253.0,0.0,1,...,0.0,0.0,0.0,2,0,1.0,0.0,1.0,0.0,0.0
4,5,168173265,168174026,-215,0.0,0.0,0,0.0,0.0,0,...,0.0,0.0,0.0,2,0,0.0,0.9,0.1,0.0,0.0


In [131]:
pd.value_counts(X6['model_pred_label'].values, sort=False)

 1.0    962
 0.0    550
-1.0      1
 2.0      2
dtype: int64

In [132]:
X6 = X6[X6.model_pred_label != -1]
pd.value_counts(X6['model_pred_label'].values, sort=False)

1.0    962
0.0    550
2.0      2
dtype: int64

In [133]:
pd.value_counts(X6['GIAB_Crowd'].values, sort=False)

0      14
1    1162
2     338
dtype: int64

In [134]:
X6.to_csv('X6_df.csv', index=False)

In [135]:
# Calculate Precision Score
true = X6['GIAB_Crowd']
predicted = X6['model_pred_label']
precision_score(true, predicted, average='micro') 

0.41941875825627478

In [136]:
from sklearn.metrics import confusion_matrix
ytest = X6['GIAB_Crowd']
predict = X6['model_pred_label']
print(confusion_matrix(ytest, predict))

[[  0  14   0]
 [527 634   1]
 [ 23 314   1]]


In [137]:
pd.crosstab(ytest, predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,2.0,All
True,,,,
0,0,14,0,14
1,527,634,1,1162
2,23,314,1,338
All,550,962,2,1514


In [82]:
output_notebook()

Loading BokehJS ...

In [83]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X6, values='Het_Var', title='Heterozygous Variant Labels', color='LightSlateGray', bins=30, xlabel="Predict Probability", ylabel="Frequency")
output_file("pred_prob_het_var.html")
show(p)

In [84]:
# Plot the counts of each predicted probability for Hom Var events
p = figure()
p = Histogram(X6, values='Hom_Var', title='Homozygous Variant Labels', color='LightSlateGray', bins=30, xlabel="Predict Probability", ylabel="Frequency")
output_file("pred_prob_hom_var.html")
show(p)

In [85]:
pd.value_counts(X6['GIAB_Crowd'].values, sort=False)

0     598
2     473
1    1734
dtype: int64

In [86]:
pd.value_counts(X6['model_pred_label'].values, sort=False)

1.0    1907
0.0     898
dtype: int64

In [89]:
d = {'Label' : [0, 0, 1, 1, 2, 2],
     'Source' : ['Pred', 'Actual', 'Pred', 'Actual','Pred', 'Actual'],
    'Count' : [898, 598, 1907, 1734, 0, 473],}
df = pd.DataFrame(d)
df = df[['Label', 'Source', 'Count']]
df

,Label,Source,Count
0,0,Pred,898
1,0,Actual,598
2,1,Pred,1907
3,1,Actual,1734
4,2,Pred,0
5,2,Actual,473


In [90]:
p = Bar(df, label='Label', xlabel="Genotype", ylabel="Count", values='Count', group='Source',plot_width=900, 
        plot_height=650, title="Predicted vs Actual", legend='top_left')

p.legend.legend_spacing = 0
show(p)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning:

legend_spacing was deprecated in Bokeh 0.12.3 and will be removed, use Legend.spacing instead.



***
Prediction : '-1' Values
***

In [91]:
# Read in HG002 DEL dataframe
HG002_pred_min_1 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/svviz.Annotate.DEL.HG002_minus_one.csv')

In [92]:
# Read in HG002 DEL dataframe
HG002_pred_min_1_ = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/svviz.Annotate.DEL.HG002_minus_one.csv')

In [93]:
### Drop irrelevant columns
# Note: Features in the prediction dataframe must match the features in the dataframe (CrowdVar) used to train the RF model
# NOTE TODO: Include size
HG002_pred_min_1.drop(['GTcons'], axis=1, inplace = True)
HG002_pred_min_1.drop(['GTconflict'], axis=1, inplace = True)
HG002_pred_min_1.drop(['GTsupp'], axis=1, inplace = True)
HG002_pred_min_1.drop('SVtype', axis=1)
HG002_pred_min_1.drop('type',axis=1)
HG002_pred_min_1.drop(['type'],axis=1, inplace = True)
HG002_pred_min_1.drop(['SVtype'],axis=1, inplace = True)
HG002_pred_min_1.drop(['start'],axis=1, inplace = True)
HG002_pred_min_1.drop(['end'],axis=1, inplace = True)
HG002_pred_min_1.drop(['chrom'],axis=1, inplace = True)
HG002_pred_min_1.drop(['Size'],axis=1, inplace = True)
HG002_pred_min_1.drop(['TenX.GT'],axis=1, inplace = True)
HG002_pred_min_1.drop(['pacbio.GT'],axis=1, inplace = True)
HG002_pred_min_1.drop(['IllMP.GT'],axis=1, inplace = True)
HG002_pred_min_1.drop(['Ill250.GT'],axis=1, inplace = True)
HG002_pred_min_1.drop(['Ill300x.GT'],axis=1, inplace = True)
HG002_pred_min_1.drop(['sample'],axis=1, inplace = True)
HG002_pred_min_1.drop(['id'],axis=1, inplace = True)
# HG002_pred.drop(['GIAB_Crowd'],axis=1, inplace = True)

In [94]:
HG002_pred_min_1.head()

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,pacbio.ref_insertSize_mean,pacbio.ref_insertSize_std,pacbio.ref_reason_alignmentScore,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,576.336245,6.124298,229.0,572.580786,146.677196,229.0,0.0,0.0,536.283903,92.909580,...,0.0,0.000000,0.0,0,0.0,0,0.0,0,0,-1
1,570.325000,3.157432,40.0,620.175000,133.582350,39.0,1.0,0.0,529.624239,92.199274,...,10741.9,3922.716723,20.0,0,0.0,0,0.0,0,0,-1
2,574.447761,11.021395,134.0,552.843284,155.619794,133.0,1.0,0.0,535.542208,85.234603,...,8703.0,0.000000,1.0,0,0.0,1,1.0,0,0,-1
3,580.686274,10.901502,51.0,892.529412,257.208800,27.0,24.0,0.0,534.831862,87.645697,...,11276.3,4110.799911,20.0,0,0.0,1,1.0,0,0,-1
4,561.728814,20.434957,59.0,791.067797,236.326610,53.0,6.0,0.0,487.928712,98.914340,...,13458.0,2863.866757,5.0,0,0.0,0,0.0,0,0,-1


In [95]:
HG002_pred_min_1.to_csv('HG002_pred_min_1.csv', index=False)

In [96]:
X3 = HG002_pred_min_1

** Impute missing values using KNN **

In [97]:
# Convert dataframe to matrix
X3=X3.as_matrix()

#Imput missing values from three closest observations
X3_imputed=KNN(k=3).complete(X3)
X3=pd.DataFrame(X3_imputed)

Imputing row 1/1191 with 1 missing, elapsed time: 1.128
Imputing row 101/1191 with 2 missing, elapsed time: 1.148
Imputing row 201/1191 with 1 missing, elapsed time: 1.173
Imputing row 301/1191 with 1 missing, elapsed time: 1.188
Imputing row 401/1191 with 1 missing, elapsed time: 1.201
Imputing row 501/1191 with 1 missing, elapsed time: 1.222
Imputing row 601/1191 with 1 missing, elapsed time: 1.231
Imputing row 701/1191 with 2 missing, elapsed time: 1.250
Imputing row 801/1191 with 2 missing, elapsed time: 1.258
Imputing row 901/1191 with 1 missing, elapsed time: 1.268
Imputing row 1001/1191 with 1 missing, elapsed time: 1.277
Imputing row 1101/1191 with 1 missing, elapsed time: 1.292


In [98]:
# Test whether values are contained in 1 array and not the other
a = ['Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_insertSizeScore_orientation','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_alignmentScore','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_insertSizeScore_insertSizeScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_insertSizeScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_insertSizeScore_insertSizeScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_insertSizeScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','GIAB_Crowd']
b = ['chrom','start','end','sample','id','type','SVtype','Size','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_insertSizeScore_orientation','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_alignmentScore','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_insertSizeScore_insertSizeScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_insertSizeScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_insertSizeScore_insertSizeScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_insertSizeScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct']
set(a) - set(b)

{'GIAB_Crowd'}

In [99]:
# Rename Columns in the dataframe
X3.columns = ['Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_insertSizeScore_orientation','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_alignmentScore','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_insertSizeScore_insertSizeScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_insertSizeScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_insertSizeScore_insertSizeScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_insertSizeScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','GIAB_Crowd']

In [100]:
# Select columns that match columns in training dataframe
#Features in training set must match features in the prediction set
X3=X3[['Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct']]

In [101]:
X3.to_csv('X3.csv', index=False)

** Standardize Data **

In [102]:
# scaler=preprocessing.StandardScaler()
# X=scaler.fit_transform(X3)

In [103]:
model.predict(X)

array([ 0.,  1.,  1., ...,  0.,  1.,  1.])

** Precision Score **

Data: 5000 randomly selected datapoints (Round 1) svviz GT vs. model selected GT from trained RF model

The ratio of tp /(tp +fn)

Intuitively the ability of a model to avoid labeling negative events as positive events

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

In [104]:
model.predict_proba(X)

array([[ 0. ,  0.7,  0.3,  0. ],
       [ 0. ,  0.3,  0.7,  0. ],
       [ 0. ,  0.4,  0.6,  0. ],
       ..., 
       [ 0. ,  0.5,  0.5,  0. ],
       [ 0. ,  0.4,  0.6,  0. ],
       [ 0. ,  0.4,  0.6,  0. ]])

In [105]:
pred = model.predict_proba(X)

In [106]:
X6 = pd.concat([X3, pd.DataFrame(pred, columns=['1','2', '3', '4'])])

In [108]:
X6.to_csv('X6_3_.csv', index=False)

In [127]:
X6 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant_Analysis/X6_3_.csv')

In [128]:
X6.head()

,1,2,3,4,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,pacbio.ref_count,pacbio.ref_insertSize_mean,pacbio.ref_insertSize_std,pacbio.ref_reason_alignmentScore,refN_cnt,refN_pct,segdup_cnt,segdup_pct,tandemrep_cnt,tandemrep_pct
0,0.0,0.7,0.3,0.0,969.740000,19.247660,50.0,422.060000,87.064898,50.0,...,0.0,0.0,0.000000,0.0,0,0,0,0.0,0,0.0
1,0.0,0.3,0.7,0.0,952.000000,11.000000,4.0,487.250000,37.552463,4.0,...,20.0,10741.9,3922.716723,20.0,0,0,0,0.0,0,0.0
2,0.0,0.4,0.6,0.0,960.069767,17.710062,43.0,412.302326,70.912175,43.0,...,1.0,8703.0,0.000000,1.0,0,0,1,1.0,0,0.0
3,0.0,0.5,0.5,0.0,929.428571,28.833795,7.0,484.142857,94.816256,7.0,...,20.0,11276.3,4110.799911,20.0,0,0,1,1.0,0,0.0
4,0.0,0.4,0.6,0.0,896.000000,6.000000,2.0,584.500000,294.500000,2.0,...,5.0,13458.0,2863.866757,5.0,0,0,0,0.0,0,0.0


In [113]:
X6.shape

(1191, 174)

In [129]:
X6['GIAB_Crowd'] = HG002_pred_min_1['GIAB_Crowd']
X6['model_pred_label'] = model.predict(X)
X6['chrom'] = HG002_pred_min_1_['chrom']
X6['start'] = HG002_pred_min_1_['start']
X6['end'] = HG002_pred_min_1_['end']
X6['Size'] = HG002_pred_min_1_['Size']
X6['GTcons'] = HG002_pred_min_1_['GTcons']
X6['Ill250.GT'] = HG002_pred_min_1_['Ill250.GT']
X6['Ill300x.GT'] = HG002_pred_min_1_['Ill300x.GT']
X6['IllMP.GT'] = HG002_pred_min_1_['IllMP.GT']
X6['pacbio.GT'] = HG002_pred_min_1_['pacbio.GT']

In [115]:
X6.shape

(1191, 185)

In [130]:
X6.head()

,1,2,3,4,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,model_pred_label,chrom,start,end,Size,GTcons,Ill250.GT,Ill300x.GT,IllMP.GT,pacbio.GT
0,0.0,0.7,0.3,0.0,969.740000,19.247660,50.0,422.060000,87.064898,50.0,...,0.0,1,33901563,33901589,-25,-1,-1.0,-1.0,-1.0,2.0
1,0.0,0.3,0.7,0.0,952.000000,11.000000,4.0,487.250000,37.552463,4.0,...,1.0,1,232503945,232503984,-38,-1,-1.0,-1.0,-1.0,1.0
2,0.0,0.4,0.6,0.0,960.069767,17.710062,43.0,412.302326,70.912175,43.0,...,1.0,1,238110204,238110233,-28,-1,-1.0,-1.0,-1.0,-1.0
3,0.0,0.5,0.5,0.0,929.428571,28.833795,7.0,484.142857,94.816256,7.0,...,0.0,1,248834852,248835006,-153,-1,-1.0,-1.0,-1.0,1.0
4,0.0,0.4,0.6,0.0,896.000000,6.000000,2.0,584.500000,294.500000,2.0,...,1.0,10,113191337,113191405,-67,-1,-1.0,-1.0,-1.0,1.0


In [117]:
X6.to_csv('X6_all.csv', index=False)

In [131]:
X6.rename(columns={'1': 'unknown'}, inplace=True)
X6.rename(columns={'2': 'Hom_Var'}, inplace=True)
X6.rename(columns={'3': 'Het_Var'}, inplace=True)
X6.rename(columns={'4': 'Hom_Ref'}, inplace=True)

In [132]:
X6.head()

,unknown,Hom_Var,Het_Var,Hom_Ref,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,model_pred_label,chrom,start,end,Size,GTcons,Ill250.GT,Ill300x.GT,IllMP.GT,pacbio.GT
0,0.0,0.7,0.3,0.0,969.740000,19.247660,50.0,422.060000,87.064898,50.0,...,0.0,1,33901563,33901589,-25,-1,-1.0,-1.0,-1.0,2.0
1,0.0,0.3,0.7,0.0,952.000000,11.000000,4.0,487.250000,37.552463,4.0,...,1.0,1,232503945,232503984,-38,-1,-1.0,-1.0,-1.0,1.0
2,0.0,0.4,0.6,0.0,960.069767,17.710062,43.0,412.302326,70.912175,43.0,...,1.0,1,238110204,238110233,-28,-1,-1.0,-1.0,-1.0,-1.0
3,0.0,0.5,0.5,0.0,929.428571,28.833795,7.0,484.142857,94.816256,7.0,...,0.0,1,248834852,248835006,-153,-1,-1.0,-1.0,-1.0,1.0
4,0.0,0.4,0.6,0.0,896.000000,6.000000,2.0,584.500000,294.500000,2.0,...,1.0,10,113191337,113191405,-67,-1,-1.0,-1.0,-1.0,1.0


In [133]:
X6=X6[['chrom','start','end','Size','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','refN_cnt','refN_pct','segdup_cnt','segdup_pct','tandemrep_cnt','tandemrep_pct','Ill250.GT','Ill300x.GT','IllMP.GT','pacbio.GT','GIAB_Crowd','GTcons','model_pred_label','unknown','Hom_Ref','Hom_Var','Het_Var']]

In [134]:
X6.head()

,chrom,start,end,Size,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,Ill300x.GT,IllMP.GT,pacbio.GT,GIAB_Crowd,GTcons,model_pred_label,unknown,Hom_Ref,Hom_Var,Het_Var
0,1,33901563,33901589,-25,969.740000,19.247660,50.0,422.060000,87.064898,50.0,...,-1.0,-1.0,2.0,-1,-1,0.0,0.0,0.0,0.7,0.3
1,1,232503945,232503984,-38,952.000000,11.000000,4.0,487.250000,37.552463,4.0,...,-1.0,-1.0,1.0,-1,-1,1.0,0.0,0.0,0.3,0.7
2,1,238110204,238110233,-28,960.069767,17.710062,43.0,412.302326,70.912175,43.0,...,-1.0,-1.0,-1.0,-1,-1,1.0,0.0,0.0,0.4,0.6
3,1,248834852,248835006,-153,929.428571,28.833795,7.0,484.142857,94.816256,7.0,...,-1.0,-1.0,1.0,-1,-1,0.0,0.0,0.0,0.5,0.5
4,10,113191337,113191405,-67,896.000000,6.000000,2.0,584.500000,294.500000,2.0,...,-1.0,-1.0,1.0,-1,-1,1.0,0.0,0.0,0.4,0.6


In [135]:
true = X6['GIAB_Crowd']
pred = X6['model_pred_label']
precision_score(true, pred, average='micro') 

0.0

In [136]:
X6.to_csv('X6_minus_one.csv', index=False)

In [137]:
from sklearn.metrics import confusion_matrix
ytest = X6['GIAB_Crowd']
predict = X6['model_pred_label']
print(confusion_matrix(ytest, predict))

[[  0 427 764]
 [  0   0   0]
 [  0   0   0]]


In [138]:
pd.crosstab(ytest, predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
-1,427,764,1191
All,427,764,1191


In [139]:
output_notebook()

Loading BokehJS ...

In [140]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X6, values='Het_Var', title='Heterozygous Variant Labels', color='LightSlateGray', bins=30, xlabel="Predict Probability", ylabel="Frequency")
output_file("pred_prob_het_var_min1.html")
show(p)

In [141]:
# Plot the counts of each predicted probability for Hom Var events
p = figure()
p = Histogram(X6, values='Hom_Var', title='Homozygous Variant Labels', color='LightSlateGray', bins=30, xlabel="Predict Probability", ylabel="Frequency")
output_file("pred_prob_hom_var_min1.html")
show(p)

### ToDo: 
- Manually Ispect to determine number of correct predictions
- Report feature importance
- Manually Inspect Sites
   - Mark easy vs. non sites